In [1]:
import numpy as np
from data_loader import load_data, DATA_PATH_TEST, DATA_PATH_TRAIN, DATA_PATH_SAMPLE_SUBMISSION_TEST
import logistic_regression
import gradient_descent
import matplotlib.pyplot as plt
from label_predictor import predict_labels

In [2]:
y, tx, ids_train = load_data(DATA_PATH_TRAIN)

In [3]:
def build_k_indices(y, k_fold, seed):
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval] for k in range(k_fold)]
    return np.array(k_indices)

In [44]:
def cross_validation(y, x, k_indices, k, gamma, alpha):
    # get k'th subgroup in test, others in train
    test_indice = k_indices[k]
    train_indice = k_indices[~(np.arange(k_indices.shape[0]) == k)]
    train_indice = train_indice.reshape(-1)
    y_test = y[test_indice]
    y_train = y[train_indice]
    x_test = x[test_indice]
    x_train = x[train_indice]
    
    w, loss_train = logistic_regression.regularized_logistic_regression_gradient_descent(y_train, x_train, gamma, 50,alpha)
    loss_test = logistic_regression.calculate_loss(y_test, x_test, w)
    y_pred = predict_labels(w, x_test)
    counter = 0
    y_pred = [0 if x==-1 else x for x in y_pred]
    for x in range(y_test.shape[0]):
        if y_pred[x] == y_test[x]:
            counter += 1
    percent = 100*counter/y_test.shape[0]
#     print(percent)
    return loss_train, loss_test, w, percent

In [5]:
def build_poly(tx, degree):
    for idx, x in enumerate(tx.T):
        if idx == 0:
            arr_out = build_poly_one_column(x, degree)
        else:
            arr_out = np.c_[arr_out, build_poly_one_column(x, degree)]
    return arr_out

In [6]:
def build_poly_one_column(x, degree):
    arr = np.zeros((x.shape[0], degree+1))
    for degre in range(degree+1):
        arr[:,degre] = np.power(x, degre)
    return arr

In [22]:
def cross_validation_demo():
    seed = 12
    k_fold = 4
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    # define lists to store the loss of training data and test data
    loss_tr = []
    loss_te = []
    # cross validation
    loss_tr_tmp = []
    loss_te_tmp = []
    tx_train = np.delete(tx, [5, 12, 15, 18, 19, 20, 21, 23, 25, 27, 28, 29, 30], axis=1)
    tx_train = build_poly(tx_train, 3)
    for k in range(k_fold):
        loss_tr, loss_te,_ = cross_validation(y, tx_train, k_indices, k)
        loss_tr_tmp.append(loss_tr)
        loss_te_tmp.append(loss_te)
        print("the loss of the training set is: ", np.mean(loss_tr_tmp))
        print("the loss of the test set is: ", np.mean(loss_te_tmp))

cross_validation_demo()


ValueError: operands could not be broadcast together with shapes (26123,) (2952,) 

In [45]:
def cross_validation_search_param(tx, y):
    seed = 12
    k_fold = 4
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    
    #Best parameter
    bestW = []
    bestRatio = 0
    bestG = 0
    bestA = 0
    bestD = 0
    #Test of different degrees
    count = 0
    for d in np.arange(2, 13, 1):  
        tx_train = build_poly(tx, d)
        for a in np.arange(0.1, 1, 0.1):
            for g in np.arange(0.1, 1, 0.1):
                # define lists to store the ratio of true mapping
                ratio = 0
                ratios= []
                for k in range(k_fold):
                    _, _, w, ratio = cross_validation(y, tx_train, k_indices, k, g, a)
                    ratios.append(ratio)
                if np.mean(ratios) > bestRatio:
                    bestW = w
                    bestG = g
                    bestA = a
                    bestD = d
                    bestRatio = np.mean(ratios)
                count += 1
#                 if(count%100 == 0):
#                     print(count)
#                 print("ratio:", np.mean(ratios))
    print("bestRatio:", bestRatio)
    return bestW, bestG, bestA, bestD
counter = 0
for y_test, tx_test, id_test in zip(y, tx, ids_train):
    bestW, bestG, bestA, bestD = cross_validation_search_param(tx_test, y_test)
    print("Set ", counter)
    print("bestW ", bestW)
    print("bestG ", bestG)
    print("bestA ", bestA)
    print("bestD ", bestD)
    counter += 1

D:\Users\benja\git\MLProjet\epfml\src\logistic_regression.py:8: RuntimeWarning: divide by zero encountered in log
  return -(y.T.dot(np.log(sigma)) + (1 - y).T.dot(np.log(1 - sigma))) / len(sigma)


bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.1156202143951
bestRatio: 74.11562021439511
bestRatio: 74.11562

bestRatio: 56.402116402116405
bestRatio: 56.402116402116405
bestRatio: 56.402116402116405
bestRatio: 56.402116402116405
bestRatio: 56.402116402116405
bestRatio: 56.402116402116405
bestRatio: 56.402116402116405
bestRatio: 56.402116402116405
bestRatio: 56.402116402116405
bestRatio: 56.402116402116405
bestRatio: 56.402116402116405
bestRatio: 56.402116402116405
bestRatio: 56.402116402116405
Set  1
bestW  [-2.10002751e+00 -2.10002751e+00 -2.10002751e+00 -2.10002751e+00
 -2.10002751e+00 -2.10002751e+00 -2.10002751e+00 -2.10002751e+00
 -2.10002751e+00 -2.10002751e+00 -2.10002751e+00 -2.10002751e+00
 -2.10002751e+00 -2.10002751e+00  3.93581004e-01 -4.64069422e-01
  6.98373057e-01  6.21592253e+00  5.14031777e+01  3.95190007e+02
  2.88298297e+03  1.92713521e+04  1.03028402e+05  1.41213252e+05
 -7.88992468e+06 -1.80754315e+08 -2.10002751e+00  2.18703745e-01
 -1.49049396e+00 -1.91550796e+00 -9.26063937e+00 -4.30133336e+01
 -2.51136782e+02 -1.65351136e+03 -1.23312877e+04 -1.03383812e+05
 -9.5904582

bestRatio: 72.5609756097561
bestRatio: 72.5609756097561
bestRatio: 72.5609756097561
bestRatio: 72.5609756097561
bestRatio: 72.5609756097561
bestRatio: 72.5609756097561
bestRatio: 72.5609756097561
bestRatio: 72.5609756097561
bestRatio: 72.5609756097561
bestRatio: 72.5609756097561
bestRatio: 72.5609756097561
bestRatio: 72.5609756097561
bestRatio: 72.5609756097561
bestRatio: 72.69647696476966
bestRatio: 72.69647696476966
bestRatio: 72.69647696476966
bestRatio: 72.69647696476966
bestRatio: 72.69647696476966
bestRatio: 72.69647696476966
bestRatio: 72.69647696476966
bestRatio: 72.69647696476966
bestRatio: 72.69647696476966
bestRatio: 72.69647696476966
bestRatio: 72.69647696476966
bestRatio: 72.89972899728998
bestRatio: 72.89972899728998
bestRatio: 72.89972899728998
bestRatio: 72.89972899728998
bestRatio: 72.89972899728998
bestRatio: 72.89972899728998
bestRatio: 72.89972899728998
bestRatio: 72.89972899728998
bestRatio: 72.89972899728998
bestRatio: 72.89972899728998
bestRatio: 72.8997289972899

bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio: 40.621781319455735
bestRatio:

bestRatio: 59.64704201200343
bestRatio: 59.64704201200343
bestRatio: 59.64704201200343
bestRatio: 59.64704201200343
bestRatio: 59.64704201200343
bestRatio: 59.64704201200343
bestRatio: 59.64704201200343
bestRatio: 59.64704201200343
bestRatio: 59.64704201200343
bestRatio: 59.64704201200343
bestRatio: 59.64704201200343
bestRatio: 59.64704201200343
bestRatio: 59.64704201200343
bestRatio: 59.64704201200343
bestRatio: 59.6498999714204
Set  5
bestW  [ 1.29848127e+01  1.29848127e+01  1.29848127e+01  1.29848127e+01
  1.29848127e+01  1.29848127e+01  1.29848127e+01  1.29848127e+01
  1.29848127e+01  1.29848127e+01  1.29848127e+01  1.29848127e+01
  1.29848127e+01  1.29848127e+01 -5.53079379e+00  8.35368081e+00
 -4.99290291e+00  2.11442025e+01  8.70079323e+01  7.50473226e+02
  5.67098248e+03  4.21636593e+04  2.87577238e+05  1.60306454e+06
  3.58580612e+06 -8.84109427e+07  1.29848127e+01  3.60697271e+00
  2.02169400e+00  1.66421330e+01  1.59492055e+02  1.90453506e+03
  2.29859947e+04  2.68778268e+05

bestRatio: 64.49429510732934
bestRatio: 64.49912976213498
bestRatio: 64.49912976213498
bestRatio: 64.49912976213498
bestRatio: 64.49912976213498
bestRatio: 64.49912976213498
bestRatio: 64.49912976213498
bestRatio: 64.49912976213498
bestRatio: 64.49912976213498
bestRatio: 64.49912976213498
bestRatio: 64.49912976213498
bestRatio: 64.49912976213498
bestRatio: 64.49912976213498
bestRatio: 64.49912976213498
bestRatio: 64.49912976213498
bestRatio: 64.49912976213498
bestRatio: 64.50396441694063
bestRatio: 64.50396441694063
bestRatio: 64.50396441694063
bestRatio: 64.50396441694063
bestRatio: 64.50396441694063
bestRatio: 64.50396441694063
bestRatio: 64.50396441694063
bestRatio: 64.50396441694063
bestRatio: 64.50396441694063
bestRatio: 64.50396441694063
bestRatio: 64.50396441694063
bestRatio: 64.50396441694063
bestRatio: 64.50396441694063
bestRatio: 64.50396441694063
bestRatio: 64.50396441694063
bestRatio: 64.50396441694063
bestRatio: 64.50396441694063
bestRatio: 64.50396441694063
bestRatio: 64.